In [1]:
!pip install --upgrade openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os
import openai


In [3]:
#INPUT API
os.environ['OPENAI_API_KEY'] =  ""

# new
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

#Multi-label classification

Prompt for multi-label classification with  GPT-3.5.


In [4]:
#Define tags
tags = """
Category: “course logistics and fit”
Description: course delivery (policy, support), cost, difficulty, time commitment, grading, credit, schedule, user fit, access, background (e.g., prereqs and appropriateness of course level).
Category: “curriculum”
Description: course content, curriculum, specific topics, course structure. This focuses on the content and the pedagogical structure of the content, including flow and organization. This also includes applied material such as clinical cases and case studies. Includes references to pre-recorded discussions between experts or between a doctor and a patient. Includes specific suggestions for additional courses or content.

"""

# Define the prompt
prompt = f"""
You are a highly-skilled assistant that classifies student course feedback comments. You respond only with a JSON object.
You will be provided with a comment from a student course feedback survey. Categorize the comment with as many of the following categories as apply:

{tags}

Think step-by-step to arrive at a correct classification. Include your reasoning behind every assigned category in the output.

"""

# List of student course feedback comments
comments = [
    "It is great victory to engage with biochemistry and get out understand it by handling its concept but not drawn by the details. I believe adding the molecular biology and molecular techniques basic concepts to the courses will had extra value and can be fit into the story.",
    "The structure of this course is just great. However, I would love to have the chance to repeat all the modules as I am from a very different background.",
    # Add more comments as needed
    ]



# Classify comments using GPT-3.5-turbo
results = []
for i, comment in enumerate(comments, start=1):

    model = "gpt-3.5-turbo"  # Sử dụng mô hình GPT-4
    max_tokens = 1000  # Điều chỉnh theo nhu cầu
    response_format={ "type": "json_object" },
    response = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content":comment}
      ],
    max_tokens=max_tokens,
    temperature = 0,
)
    print(response.choices[0].message.content)





{
  "curriculum": "The comment mentions adding molecular biology and molecular techniques basic concepts to the courses, which suggests a suggestion for additional course content.",
  "course logistics and fit": "The comment also mentions that the addition of these concepts would add extra value and fit into the story, indicating a consideration of course fit and appropriateness."
}
{
  "categories": [
    "course logistics and fit",
    "curriculum"
  ],
  "reasoning": {
    "course logistics and fit": "The comment mentions the desire to repeat modules due to being from a different background. This indicates a concern about the appropriateness of the course level and user fit.",
    "curriculum": "The comment praises the structure of the course, indicating satisfaction with the course content and pedagogical structure."
  }
}


#Multi-class classification



In [7]:
tags = """
Category: “course logistics and fit”
Description: course delivery (policy, support), cost, difficulty, time commitment, grading, credit, schedule, user fit, access, background (e.g., prereqs and appropriateness of course level).
Category: “curriculum”
Description: course content, curriculum, specific topics, course structure. This focuses on the content and the pedagogical structure of the content, including flow and organization. This also includes applied material such as clinical cases and case studies. Includes references to pre-recorded discussions between experts or between a doctor and a patient. Includes specific suggestions for additional courses or content.

"""

goal_focus= "suggestions for improvement"


# Prepare the prompt

prompt = f"""
You are a highly-skilled assistant that categorizes {goal_focus} from student course feedback comments. You respond only with a JSON array.
You will be provided with an array of excerpts from a student course feedback survey. Categorize each excerpt with exactly one of the following categories:
{tags}.

Think step-by-step to arrive at a correct classification. Include your reasoning in the output.

"""

comment = "It is great victory to engage with biochemistry and get out understand it by handling its concept but not drawn by the details. I believe adding the molecular biology and molecular techniques basic concepts to the courses will had extra value and can be fit into the story."

model = "gpt-3.5-turbo-1106"  # Sử dụng mô hình GPT-4
max_tokens = 2100  # Điều chỉnh theo nhu cầu
response = client.chat.completions.create(
      model=model,
      messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content":comment}
      ],
    temperature = 0,
    max_tokens=max_tokens
)
print(response.choices[0].message.content)

[
  {
    "curriculum": "The student is suggesting adding molecular biology and molecular techniques basic concepts to the course, which falls under curriculum as it pertains to course content and specific topics."
  }
]


# Binary classification

In [14]:

# Define the prompt variables
delimiter = "####"  # Ký tự phân cách dùng để phân định rõ ràng nhận xét
question = "‘Please describe the best parts of this course" #Câu hỏi khảo sát
goal_focus = "suggestions for new topics or content"  #“feedback about the teaching or explanations”



# Define the prompt
prompt = f"""
You are a highly-skilled assistant that classifies student course feedback comments
based on whether or not they contain {goal_focus}. You answer only with a JSON
object and nothing else.
You will be provided with a comment from a student course feedback survey. The
comment will be delimited by {delimiter} characters. Each original comment was
in response to the question: "{question}". Classify the comment as either containing {goal_focus} or not containing {goal_focus}. Think step-by-step to arrive at a
correct classification. Include your reasoning in the output.
Only use yes or no for the classification. If something is N/A, use no.
"""

comments = [
    "Use of a tool like Cerego to accompany the course. Additional questions that require calculations, and more questions overall to reinforce concepts. Reduce cost.",
     "It is great victory to engage with biochemistry and get out understand it by handling its concept but not drawn by the details. I believe adding the molecular biology and molecular techniques basic concepts to the courses will had extra value and can be fit into the story."
    # Add more comments as needed
]


#For each prompt, there was also an associated function schema that was used as part of function calling to guide the format of the output
function_schema = [
{
  "name": "store_binary_classification_with_reasoning",
  "description": "Store the binary classification and reasoning of an online course survey comment in a database.",
  "parameters": {
    "type": "object",
    "properties": {
      "classification": {
        "type": "string",
        "enum": ["yes", "no"],
        "description": f"Whether or not the comment contains the {goal_focus}."
      },
      "reasoning": {
        "type": "string",
        "description": "The reasoning for the classification."
      }
    },
    "required": ["classification", "reasoning"]
  }
}
]

for i, comment in enumerate(comments, start=1):

    model = "gpt-3.5-turbo"  # Sử dụng mô hình GPT-4
    max_tokens = 2048  # Điều chỉnh theo nhu cầu
    response = client.chat.completions.create(
      model=model,
      messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": f"{delimiter}{comment}{delimiter}"}
      ],
    max_tokens=max_tokens,
    temperature = 0,

    functions=function_schema,
    function_call="auto",

)
    print(response.choices[0].message.content)






{
  "classification": "yes",
  "reasoning": "The comment contains suggestions for new topics or content. The student suggests using a tool like Cerego to accompany the course, adding additional questions that require calculations, and more questions overall to reinforce concepts. The student also suggests reducing the cost of the course."
}
None


#Extraction

In [13]:


# Define the prompt variables
delimiter = "####"  # Change this to your specific delimiter
question = "Please describe the best parts of this course?"
goal_focus = "feedback about the teaching or explanations" #“suggestions for new topics or content

# Define the prompt
prompt = f"""
You are a highly-skilled assistant that extracts {goal_focus} from student course
feedback comments. You respond only with a JSON array.\n
You will be provided with a comment from a student course feedback survey. The
comment will be delimited by {delimiter} characters. Each original comment was
in response to the question: "{question}". However, your task is to only select
excerpts which pertain to the goal focus: "{goal_focus}". Excerpts should only
be exact quotes taken from the comment; do not add or alter words under any
circumstances.\n
If you cannot extract excerpts for any reason, for example if the comment is not
relevant to the question, respond only with an empty JSON array: []\n
If there are relevant excerpts, ensure that excerpts contain all relevant text needed
to interpret them - in other words don’t extract small snippets that are missing
important context.\n
Before finalizing excerpts, review your excerpts to see if any consecutive excerpts
are actually about the same suggestion or part of the same thought. If so, combine
them into a single excerpt.

"""

# Replace this with an actual comment from the user survey
comments = [
    "It is great victory to engage with biochemistry and get out understand it by handling its concept but not drawn by the details. I believe adding the molecular biology and molecular techniques basic concepts to the courses will had extra value and can be fit into the story.",
    "Use of a tool like Cerego to accompany the course. Additional questions that require calculations, and more questions overall to reinforce concepts. Reduce cost."
    # Add more comments as needed
]


# Classify comments using GPT-3.5-turbo
results = []
for i, comment in enumerate(comments, start=1):

    model = "gpt-3.5-turbo"  # Sử dụng mô hình GPT-4
    max_tokens = 2048  # Điều chỉnh theo nhu cầu
    response = client.chat.completions.create(
      model=model,
      messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": f"{delimiter}{comment}{delimiter}"}
      ],
    max_tokens=max_tokens,
    temperature = 0

)
    print(response.choices[0].message.content)



[{"excerpt": "I believe adding the molecular biology and molecular techniques basic concepts to the courses will had extra value and can be fit into the story."}]
[ "Use of a tool like Cerego to accompany the course.", "Additional questions that require calculations, and more questions overall to reinforce concepts.", "Reduce cost." ]


# Extraction evaluation

In [12]:
# Define the prompt variables
delimiter = "####"  # Change this to your specific delimiter
question = "Please describe the best parts of this course?"
goal_focus = "feedback about the teaching or explanations" #“suggestions for new topics or content
# Define the prompt
prompt = f"""
You are a highly-skilled assistant that evaluates how well a computer program has
extracted {goal_focus} from student course feedback comments. You respond only
with a JSON object.
You will be provided with the original comment from a student course feedback
survey and a list of the extracted excerpts. Each original comment was in response
to the question: "{question}". The computer program’s task was to select exact
quote excerpts which pertain to the goal focus: "{goal_focus}".
Your task is to evaluate how well the computer program did by answering the
following questions:
- Did the program miss any relevant excerpts? Ignore spelling, punctuation, and
capitalization changes. (yes or no)
- Did the program extract any excerpts that are ambiguous because of lack of
context? (yes or no)
- If context is missing, did the needed context exist in the original comment but
not get captured in the excerpts? If all needed context was present in the excerpts,
answer ‘na’. (yes, no, or na)
- Did the program extract any irrelevant excerpts? (yes or no)
- Did the program extract any irrelevant text that is not needed to interpret the
excerpts? (yes or no)
- Did the program extract any excerpts where {goal_focus} is only implied but not
explicit? (yes or no)
- Did the program produce any excerpts that are not exact quotes from the original
comment? (yes or no)
- Did the program extract any excerpts that are not contiguous (i.e., consecutive)
in the original comment? (yes or no)
- Are there two or more excerpts that are actually about the same idea and which
could have been combined or had one dropped? (yes or no)
Think step-by-step to arrive at correct answers. Include your reasoning behind
every question in the output.

"""

# Replace this with an actual comment and list of excerpts from the user survey
comments = [
    "It is great victory to engage with biochemistry and get out understand it by handling its concept but not drawn by the details. I believe adding the molecular biology and molecular techniques basic concepts to the courses will had extra value and can be fit into the story.",
    "Use of a tool like Cerego to accompany the course. Additional questions that require calculations, and more questions overall to reinforce concepts. Reduce cost."
    # Add more comments as needed
]

excerpt_list = """
Reduce cost.", "Additional questions that require calculations, and more questions overall to reinforce concepts.",
"Additional questions that require calculations, and more questions overall to reinforce concepts."
"""

# Classify comments using GPT-3.5-turbo
results = []
for i, comment in enumerate(comments, start=1):

    model = "gpt-3.5-turbo"  # Sử dụng mô hình GPT-4
    max_tokens = 2048  # Điều chỉnh theo nhu cầu
    response = client.chat.completions.create(
      model=model,
      messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": f"{delimiter}{comment}{delimiter}"},
    {"role": "user", "content": excerpt_list}
      ],
    max_tokens=max_tokens,
    temperature = 0
)
    print(response.choices[0].message.content)



{
  "missed_relevant_excerpts": "no",
  "ambiguous_excerpts": "no",
  "missing_context": "na",
  "irrelevant_excerpts": "no",
  "irrelevant_text": "no",
  "implied_feedback": "no",
  "not_exact_quotes": "no",
  "non_contiguous_excerpts": "no",
  "repeated_excerpts": "yes"
}
{
  "missed_relevant_excerpts": "no",
  "ambiguous_excerpts": "no",
  "missing_context": "na",
  "irrelevant_excerpts": "no",
  "irrelevant_text": "no",
  "implied_feedback": "no",
  "not_exact_quotes": "no",
  "non_contiguous_excerpts": "no",
  "repeated_excerpts": "yes"
}


#Sentiment analysis

In [10]:
comments = [
    "Implement more checkpoints that review previous material throughout the course.",
    "The course was fantastic and informative. However, I had to rewatch the videos several times to write down everything that is said. I learn best by looking at the words. The videos should come with either a transcript or written words or some sort that convey the same information.",
    "I really enjoyed the course and learned a lot of applicable information for my job. I would have like a little more time between new releases of information. It would also be nice to have a live question/answer session."
# Add more comments as needed
]

tags = """
Category: "positive"
Description: The comment is predominantly pleased or happy with the course experience.
Category: "slightly positive"
Description: The comment is slightly pleased or happy.
Category: "neutral"
Description: The comment is neither positive nor negative overall.
Category: "slightly negative"
Description: The comment is slightly displeased or unhappy.
Category: "negative"
Description: The comment is predominantly displeased or unhappy.
"""
prompt = f"""
You are a highly-skilled assistant that classifies student course feedback comments.
You respond only with a JSON object.
You will be provided with a comment from a student course feedback survey.
Categorize the comment with one of the following categories:
{tags}
Think step-by-step to arrive at a correct classification. Include your reasoning
behind every assigned category in the output.
tags {tags}
"""

for i, comment in enumerate(comments, start=1):

    model = "gpt-3.5-turbo"  # Sử dụng mô hình GPT-4
    max_tokens = 2048  # Điều chỉnh theo nhu cầu
    response = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content":comment}
      ],
    max_tokens=max_tokens,
    temperature = 0
)
    print(response.choices[0].message.content)


{
  "category": "positive",
  "reasoning": "The comment suggests that the course should implement more checkpoints to review previous material. This indicates that the student is pleased with the course and believes that additional review would enhance their learning experience."
}
{
  "category": "slightly positive",
  "reasoning": "The comment starts off by stating that the course was fantastic and informative, indicating a positive experience. However, the student mentions a minor issue with having to rewatch the videos several times to write down everything. This suggests a slight dissatisfaction. Overall, the comment leans towards being slightly positive."
}
{
  "category": "positive",
  "reasoning": "The comment starts off by expressing enjoyment and learning a lot from the course, indicating a positive experience. The mention of wanting more time between new releases of information and a live question/answer session suggests areas for improvement, but overall the comment is pred

#Inductive thematic analysis

In [11]:
# Define the prompt variables
delimiter = "####"  # Change this to your specific delimiter
question = "how did they feel about the course?"

# Define the prompt
prompt = f"""
You are a highly-skilled assistant that derives the main themes from student course
feedback comments. You respond only with a JSON array.
You will be provided with a set of comments from a student course feedback
survey. Each comment is surrounded by the delimiter {delimiter}. Each original
comment was in response to the question: "{question}". Your task is to derive the
main themes from the comments. A theme is a short phrase that summarizes a
piece of feedback that is expressed by multiple students.
Respond with a JSON array of theme objects. Each theme object should have a
‘theme description’ field which describes the theme in two sentences or less, and a
‘theme title’ field (which gives a short name for the theme in 5 words or less).
If you cannot find any themes, for example if there are no comments in the original
list, respond only with an empty JSON array: []

"""


# Replace this with an actual comment and list of excerpts from the user survey
comments = [
    "The teachers they are incredible and their fascination about this topic make it more interesting.",
    "The structure of this course is just great. However, I would love to have the chance to repeat all the modules as I am from a very different background."
    "I was really impressed with the course! The instructors are so knowledgeable, and their passion for the subject shines through in every lesson."
    "The course was fantastic! I appreciated the well-organized content, and the instructors' enthusiasm added an extra layer of excitement to the material."
    "I found the course to be quite enjoyable. The teachers were engaging, and their enthusiasm made the learning experience much more enjoyable."
    "The instructors made the course truly enjoyable. Their passion for the subject made the material more relatable and interesting."
    "I had a great experience with the course! The instructors were excellent, and their passion for the material made the content more engaging."
    "The course structure was excellent, and the teachers' enthusiasm was contagious. I thoroughly enjoyed every aspect of the learning process."
    "The teachers were amazing! Their passion for the course material was evident, and it made the learning experience more enjoyable and memorable."
    "I loved the course! The instructors' passion for the subject matter was contagious, making each session engaging and informative."
    "The course was fantastic, largely due to the incredible instructors. Their enthusiasm and knowledge enhanced the overall learning experience."
    "I had a positive experience with the course. The instructors' passion for the topic was evident, and it made the learning process much more enjoyable."
     # Add more comments as needed
]


model = "gpt-3.5-turbo"
max_tokens = 2048  # Điều chỉnh theo nhu cầu
response = client.chat.completions.create(
      model=model,
      messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": f"{delimiter}{comments}{delimiter}"}
      ],
    max_tokens=max_tokens,
    temperature = 0
)

response.choices[0].message.content





'[{"theme description": "Incredible teachers with passion", "theme title": "Passionate Teachers"}, {"theme description": "Great course structure and organization", "theme title": "Excellent Course Structure"}, {"theme description": "Engaging and enjoyable learning experience", "theme title": "Engaging Instructors"}]'